## Scraping Property Access PH through Selenium
This notebook was prepared by Adem Inovejas, Christopher Lim, Czarina Tiu, and Uriel Grace Magtibay (students of DATA102 S11 Y2022-2023).  

In your notebook, make sure you have the following details outlined:
- The website scraped
- Date and time when the data was collected
- What were the challenges encountered? You may narrate or illustrate this in the notebook.
- Do you think the collected data contains any personally identifiable information (PII)?
- Conclude with your key learnings and findings.

## Importing Libaries

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import datetime
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

## Setup

In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [3]:
url = "https://propertyaccess.ph/offer/sale"
driver.get(url)

## Data Collection

### Collecting  the URLs of each properties

In [4]:
has_pages = True
pages = 1
urls = []
total_time = 0
while(has_pages):  
    start = time.perf_counter()
    
    products_container = driver.find_element(by="xpath", value='//div[@class="list-product page-content"]')
    
    # gets the list of products
    items = products_container.find_elements(by="xpath", value='.//div[@class="product-wrapper"]')
    for item in items:
        # get the href of the anchor of "product-img"
        url = item.find_element(by="xpath", value='.//div[@class="product-img"]/a').get_attribute('href')
        date_published = item.find_element(by="xpath", value='.//div[@class="date"]').text
        urls.append((url, date_published))
    
    end = time.perf_counter()
    
    total_time += end - start
    print('Extracted Page %d | %d items found | %.2fs' % (pages, len(items), end - start))
    try:
        # checks if next page button is available by getting the xpath
        next_page = driver.find_element("xpath", value='//ul[@class="pargination"]/li[@class="next flex-end"]/a').get_attribute("href");
        driver.get(next_page)
        pages += 1
    except:
        # if next page button is not available, stop the loop
        has_pages = False

print('Number of pages:', pages)
print('Number of products:', len(urls))
print('Average Time per page:', total_time / len(urls))
print('Total Time:', total_time)

Extracted Page 1 | 20 items found | 0.60s
Extracted Page 2 | 20 items found | 0.72s
Extracted Page 3 | 20 items found | 0.75s
Extracted Page 4 | 20 items found | 0.78s
Extracted Page 5 | 20 items found | 0.64s
Extracted Page 6 | 20 items found | 0.60s
Extracted Page 7 | 20 items found | 0.74s
Extracted Page 8 | 20 items found | 0.59s
Extracted Page 9 | 20 items found | 0.73s
Extracted Page 10 | 20 items found | 0.72s
Extracted Page 11 | 20 items found | 0.82s
Extracted Page 12 | 20 items found | 0.58s
Extracted Page 13 | 20 items found | 0.71s
Extracted Page 14 | 20 items found | 0.59s
Extracted Page 15 | 20 items found | 0.74s
Extracted Page 16 | 20 items found | 0.75s
Extracted Page 17 | 20 items found | 0.59s
Extracted Page 18 | 20 items found | 0.63s
Extracted Page 19 | 20 items found | 0.72s
Extracted Page 20 | 20 items found | 0.78s
Extracted Page 21 | 20 items found | 0.59s
Extracted Page 22 | 20 items found | 0.79s
Extracted Page 23 | 20 items found | 0.80s
Extracted Page 24 | 

Exporting the URLs through pandas dataframe (as .csv)

In [5]:
df = pd.DataFrame(urls, columns=['URL', 'Date Published'])
df.head()

,URL,Date Published
0,https://propertyaccess.ph/property/3-br-condo-...,Published on: 30/03/2022
1,https://propertyaccess.ph/property/1br-condo-i...,Published on: 30/03/2022
2,https://propertyaccess.ph/property/3-bedroom-c...,Published on: 17/07/2022
3,https://propertyaccess.ph/property/3-bedroom-c...,Published on: 17/07/2022
4,https://propertyaccess.ph/property/2br-condo-i...,Published on: 1/06/2022


In [6]:
df.to_csv('property_urls.csv', index=False)

### Going through each property page to get the details

In [7]:
df = pd.read_csv('property_urls.csv')
df.head()

,URL,Date Published
0,https://propertyaccess.ph/property/3-br-condo-...,Published on: 30/03/2022
1,https://propertyaccess.ph/property/1br-condo-i...,Published on: 30/03/2022
2,https://propertyaccess.ph/property/3-bedroom-c...,Published on: 17/07/2022
3,https://propertyaccess.ph/property/3-bedroom-c...,Published on: 17/07/2022
4,https://propertyaccess.ph/property/2br-condo-i...,Published on: 1/06/2022


In [8]:
data = []
total_time = 0
for i, item in enumerate(list(zip(df['URL'], df['Date Published']))):
    try:
        start = time.perf_counter()
        url, date_published = item
        driver.get(url)

        # get the container which contains the property details
        property_details = driver.find_element(by="xpath", value='//div[@class="product-detail-wraper"]')

        # find the name
        name = property_details.find_element(by="xpath", value='.//h1[@class="basic-info__name"]').text
        address = property_details.find_element(by="xpath", value='.//div[@class="basic-info__street"]').text
        price = property_details.find_element(by="xpath", value='.//div[@class="basic-info__price"]').text
        author = property_details.find_element(by="xpath", value='.//div[@class="agent-name line-clamp lc-2"]').text

        # get the bedrooms, showers, parking, furnish type, total developed and lot area through utility info
        utility_info = property_details.find_element(by="xpath", value='.//div[@class="basic-info__utilities"]')

        try:
            bedrooms = utility_info.find_element(by="xpath", value='//img[@data-src="https://cdn.propertyaccess.ph/prod/v1.2.5-fis-hotfix-2/f24d0f0d48276caeca2060fd60abf269.svg"]/..').text
        except:
            bedrooms = None

        try:
            showers = utility_info.find_element(by="xpath", value='//img[@data-src="https://cdn.propertyaccess.ph/prod/v1.2.5-fis-hotfix-2/a6e500c94a1eaacdd59fde045fba89f0.svg"]/..').text
        except:
            showers = None

        try:
            furnish = utility_info.find_element(by="xpath", value='//img[@data-src="https://cdn.propertyaccess.ph/prod/v1.2.5-fis-hotfix-2/9fb04d2eb34997da34e3dcfd140f51b6.svg"]/..').text
        except:
            furnish = None

        try:
            parking = utility_info.find_element(by="xpath", value='//img[@data-src="https://cdn.propertyaccess.ph/prod/v1.2.5-fis-hotfix-2/33d487e03cfed78b821458ab3c39b574.svg"]/..').text
        except:
            parking = None

        try:
            total_developed = utility_info.find_element(by="xpath", value='//img[@data-src="https://cdn.propertyaccess.ph/prod/v1.2.5-fis-hotfix-2/57cebe2c8bcb0aa7f604503ddb740621.svg"]/..').text
        except:
            total_developed = None

        try:
            lot_area = utility_info.find_element(by="xpath", value='//img[@data-src="https://cdn.propertyaccess.ph/prod/v1.2.5-fis-hotfix-2/da2ffe9d4238be9a3d697b4195b64fe7.svg"]/..').text
        except:
            lot_area = None

        # if section title is Property Features or Amenities
        features = [feature.text for feature in property_details.find_elements(by="xpath", value='.//section[@class="product-feature"]//div[text()="Property Features" or text()="Amenities"]/..//div[@class="item-listing readmore-target"]//p')]

        # if section title is Facilities
        facilities = [facility.text for facility in property_details.find_elements(by="xpath", value='.//section[@class="product-feature"]//div[text()="Facilities"]/..//div[@class="item-listing readmore-target"]//p')]

        # if section title is Nearby Places
        nearby_places = [place.text for place in property_details.find_elements(by="xpath", value='.//section[@class="product-nearby"]//div[@class="item-title"]')]

        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        end = time.perf_counter()

        total_time += end - start
        
        print('[%4d] %s | %.2fs' % (i, name, end - start))

        data.append((name, address, author, price, bedrooms, showers, parking, furnish, total_developed, lot_area, features, facilities, nearby_places, url, timestamp))
    except:
        # if page is un-available
        print('Error occured on', url)

[   0] 3 BR Condo in The Grand Midori Ortigas, Pasig | 4.75s
[   1] 1 Bedroom Condo in The Grand Midori Ortigas, Pasig | 1.29s
[   2] 3 Bedroom Condo in Aurelia Residences, Taguig | 1.75s
[   3] 3 Bedroom Condo in Shang Residences at Wack Wack, Mandaluyong | 1.44s
[   4] 2BR Condo in Residences at The Galleon, Pasig | 2.14s
[   5] Penthouse in Residences at The Galleon, Pasig | 1.41s
[   6] 3 Bedroom Condo in Aurelia Residences, Taguig | 1.39s
[   7] 3 Bedroom Condo in Valencia Hills Tower E, Quezon City | 1.31s
[   8] 2 Bedroom Condo in Valencia Hills Tower E, Quezon City | 1.50s
[   9] 1 Bedroom Condo in Valencia Hills Tower E, Quezon City | 1.18s
[  10] 3 Bedroom Condo in Aurelia Residences, Taguig | 1.62s
[  11] 2 Bedroom Condo in The Rise Makati, Makati | 1.41s
[  12] 3 Bedroom Condo in Aurelia Residences, Taguig | 1.64s
[  13] 2 Bedroom Condo in Shang Residences at Wack Wack, Mandaluyong | 1.39s
[  14] 1 Bedroom Condo in Shang Residences at Wack Wack, Mandaluyong | 1.65s
[  15] 1

[ 144] 3 Bedroom Condo in One Shangri-la Place, Mandaluyong | 1.28s
[ 145] 4 Bedroom House & Lot, Las Piñas | 0.98s
[ 146] 4 Bedroom House & Lot, Las Piñas | 1.17s
[ 147] 5 Bedroom House & Lot, Las Piñas | 1.52s
[ 148] 4 Bedroom House & Lot, Parañaque | 1.35s
[ 149] 4 Bedroom House & Lot, Muntinlupa | 1.02s
[ 150] 4 Bedroom House & Lot, Muntinlupa | 1.00s
[ 151] 4 Bedroom House & Lot, Muntinlupa | 1.03s
[ 152] 4 Bedroom House & Lot, Parañaque | 1.29s
[ 153] 5 Bedroom House & Lot, Muntinlupa | 1.25s
[ 154] 4 Bedroom House & Lot, Parañaque | 1.14s
[ 155] 5 Bedroom House & Lot, Muntinlupa | 1.09s
[ 156] Residential Lot, Parañaque | 0.97s
[ 157] Clean Title One Bedroom Condo in One Uptown Residence, Taguig | 1.61s
[ 158] 2 Bedroom Condo at The Trion Towers, Taguig | 1.58s
[ 159] 3 Bedroom Condo in Antel Platinum, Makati | 1.89s
[ 160] 10 Bedroom House & Lot, Medellin | 1.06s
[ 161] 2 Bedroom Condo in Urban Deca Homes Ortigas, | 1.05s
[ 162] 2 Bedroom Condo in Urban Deca Homes Ortigas, | 0.

[ 304] 3 Bedroom House & Lot, General Trias | 1.08s
[ 305] 4 Bedroom House & Lot, Lipa | 1.03s
[ 306] 5 Bedroom House & Lot, Parañaque | 1.03s
[ 307] Residential Lot, Biñan | 0.99s
[ 308] 2 Bedroom House & Lot, Mabalacat | 1.12s
[ 309] 4 Bedroom House & Lot, Dasmariñas | 1.52s
[ 310] 4 Bedroom House & Lot, Manila | 1.08s
[ 311] 3 Bedroom House & Lot, Quezon City | 1.21s
[ 312] 4 Bedroom House & Lot, Quezon City | 1.47s
[ 313] 4 Bedroom House & Lot, Quezon City | 1.33s
[ 314] Residential Lot, Calabarzon | 1.04s
[ 315] 5 Bedroom House & Lot, Angeles | 1.18s
[ 316] Studio Condo, Cagayan de Oro | 1.19s
[ 317] 3 Bedroom Condo in Avida Cityflex Towers, Taguig | 1.70s
[ 318] 6 Bedroom House & Lot, Las Piñas | 1.95s
[ 319] 2 Bedroom Condo in Two Serendra, Taguig | 1.40s
[ 320] 3 Bedroom Condo in The Infinity Tower, Taguig | 1.80s
[ 321] 3 Bedroom Condo in Tuscany Private Estate, Taguig | 2.22s
[ 322] 2 Bedroom Condo in Avida Towers Verte, Taguig | 1.51s
[ 323] 2 Bedroom Condo in Uptown Ritz, T

[ 463] 3 Bedroom Condo in Two Serendra, Taguig | 1.65s
[ 464] 3 Bedroom Condo in Madison Park West, Taguig | 1.62s
[ 465] 4 Bedroom House & Lot, Santa Rosa | 1.30s
[ 466] Studio Condo, Lungsod ng Tagaytay | 2.37s
[ 467] 4 Bedroom House & Lot, Las Piñas | 1.93s
[ 468] 2 Bedroom House & Lot, Muntinlupa | 2.53s
[ 469] 3 Bedroom House & Lot, Las Piñas | 1.14s
[ 470] 1 Bedroom Condo in The Residences at Greenbelt, Makati | 1.29s
[ 471] 3 Bedroom Condo in Park Terraces, Makati | 1.27s
[ 472] 3 Bedroom Condo in The Twin Towers, Makati | 2.15s
[ 473] Tagaytay Cliffton Resort Suites | 1.30s
[ 474] Studio Condo in Tagaytay Cliffton Resort Suites, Quezon City | 1.46s
[ 475] 3 Bedroom Condo in The Ritz Tower, Makati | 2.20s
[ 476] 3 Bedroom Condo in Two Roxas Triangle, Makati | 1.33s
[ 477] 1 Bedroom Condo in One Serendra, Taguig | 1.33s
[ 478] 2 Bedroom Condo in Fort Palm Spring, Taguig | 2.15s
[ 479] 2 Bedroom Condo in McKinley Park Residences, Taguig | 1.82s
[ 480] 2 Bedroom Condo in The Bellag

[ 622] 1 Bedroom Condo in The Paddington Place, Mandaluyong | 2.24s
[ 623] 1 Bedroom Condo in Soho Central, Mandaluyong | 3.40s
[ 624] 2 Bedroom Condo in One Shangri-la Place, Mandaluyong | 2.69s
[ 625] 1 Bedroom Condo in Torre de Manila, Manila | 4.10s
[ 626] 3 Bedroom Condo in Bonifacio Ridge, Taguig | 1.85s
[ 627] 2 Bedroom Condo in Suntrust Treetop Villas, Mandaluyong | 3.01s
[ 628] Residential Lot in Hillcrest Estates Nuvali, Laguna | 2.38s
[ 629] 4 Bedroom House & Lot in Amadeo, Cavite | 1.27s
[ 630] 2 Bedroom Condo in Besco Residences, Pampanga | 1.30s
[ 631] 2 Bedroom House & Lot, Consolacion | 1.21s
[ 632] 1 Bedroom House & Lot, Barili | 1.35s
[ 633] 2 Bedroom Condo in One Shangri-la Place, Mandaluyong | 2.61s
[ 634] 3 Bedroom Condo in Sapphire Residences, Taguig | 1.55s
[ 635] 3 Bedroom Condo in McKinley Garden Villas, Taguig | 2.22s
[ 636] 2 Bedroom Condo in Alta Spatial, Valenzuela | 2.13s
[ 637] 2 Bedroom Condo, Parañaque | 1.48s
[ 638] Studio Condo, Quezon City | 2.88s
[ 

...

[1850] 1 bedroom in The Meridian | 0.99s
[1851] Studio Unit in Las Piñas | 1.64s
[1852] Studio Unit in Tagaytay | 1.82s
[1853] Studio Type Condominium in Morgan Executive Residence, BGC, Taguig City | 0.97s
[1854] Studio Type Condominium in Venice Luxury Residences, Mckinley Hill, Taguig City | 0.87s
[1855] Condo for Sale in Eastwood Legrand Tower 2 | 1.32s
[1856] Studio Type Condominium unit in Morgan Suites Executive Residences, Taguig | 1.07s
[1857] 4-Storey Residential Building For Sale in Makati | 0.82s
[1858] 2 BR Penthouse Unit at Aston Residences (with Parking) | 0.80s
[1859] 2BR Unit at Kai Garden Residences with Parking, Mandaluyong | 1.34s
[1860] 3 BR House and Lot in Versailles Alabang - Muntinlupa | 1.00s
[1861] 5 BR House and Lot in Versailles Alabang - Muntinlupa | 0.89s
[1862] 3 BR House and Lot in Versailles Alabang - Muntinlupa | 0.93s
[1863] 3 BR House and Lot in Versailles Alabang - Muntinlupa | 0.90s
[1864] 4 Bedroom House and Lot in Valle Verde 2, Pasig City | 0.9

[1977] 4 BR House and Lot in Magallanes Village, Makati City | 0.98s
[1978] 2 BR Condominium Unit in Park Terraces Ayala, Makati City | 1.01s
[1979] 3 Bedroom House in Marikina Heights | 1.00s
[1980] 89 SQM 3 BR House in Antipolo | 1.05s
[1981] 120 SQM Lot in Woodland Hills. San Mateo | 0.89s
[1982] 4 BR House and Lot in Hillsborough, Muntinlupa | 0.95s
[1983] 5 BR House and Lot in Ayala Southvale Sonera, Las Piñas | 1.09s
[1984] 4 BR House and Lot in Portofino Amore, Las Piñas | 0.88s
[1985] 5 BR House and Lot in BF Homes, Parañaque City | 1.14s
[1986] 4 BR House and Lot in Southbay Gardens, Parañaque City | 1.20s
[1987] 2 BR Penthouse Condo in Kasara Urban Resort Residences, Pasig | 1.39s
[1988] 1 Bedroom 56 sqm with balcony Condominium Unit in Manhattan Heights, Quezon City | 1.34s
[1989] 1 BR Unit in Manhattan Garden Cubao, Quezon City | 0.90s
[1990] 2 Bedroom Condominium in Pioneer Woodlands, Mandaluyong | 1.10s
[1991] House & Lot in Del Nacia Ville Novaliches, Quezon City | 0.86s

[2105] 1 BR Condo in Torre Lorenzo Malate The Suites | 1.18s
[2106] 1 BR Loft in Torre Lorenzo Malate The Suites | 1.42s
[2107] Studio Condo in Torre Lorenzo Malate The Suites | 1.16s
[2108] 1 BR Condo in Torre Lorenzo Malate The Suites | 1.13s
[2109] Studio Condo in Torre Lorenzo Malate The Suites | 1.26s
[2110] 2BR Condo in Grand Hyatt Manila Residences | 1.56s
[2111] 3 BR Condo with Den in Grand Hyatt Manila Residences | 1.32s
[2112] 3BR Condo in Grand Hyatt Manila Residences | 1.29s
[2113] 1 BR Condo in Sail Residences, Pasay | 1.16s
[2114] 2 Bedroom Condo in Mandaluyong | 1.36s
[2115] 9 BR House and Lot in Alabang Hills, Muntinlupa | 1.55s
[2116] 4 BR House and Lot in Hillsborough Alabang Village, Muntinlupa | 1.72s
[2117] 7 BR House and Lot in Capitol 8, Pasig | 1.47s
[2118] Studio Unit in Bellini Tower at Venice Luxury Residences, Taguig | 1.32s
[2119] 2 BR Condo in The Veranda at Arca South, Taguig City | 1.31s
[2120] 4 BR House and Lot in AFPOVAI Subdivision, Taguig City | 0.9

[2232] 2 BR Condominium unit in Fifth Avenue Place, Taguig | 1.40s
[2233] 3BR House in Princess Homes, Caloocan City | 1.03s
[2234] 6 BR House and Lot in Dasmarinas Village, Makati City | 1.09s
[2235] 2 Bedroom Condominium in Victoria de Makati, Makati City | 1.15s
[2236] 1 BR Condominium unit in Joya South, Makati City | 1.15s
[2237] 1 Bedroom Condominium in Kroma Tower, Makati City | 1.20s
[2238] 3 BR Condominium unit in The Proscenium, Makati City | 1.31s
[2239] 2 BR Condominium unit in The Proscenium, Makati CIty | 1.22s
[2240] 3 BR Condominium unit in The Proscenium, Makati City | 1.38s
[2241] 2 BR Condominium unit in One Rockwell East Tower, Makati City | 1.39s
[2242] Studio Unit in Vista GL Taft - Manila | 1.42s
[2243] Studio Unit in 878 España by Vista Residences - Manila | 1.16s
[2244] Seven Bedroom House in Greenwoods, Pasig | 1.02s
[2245] 4 Bedroom House in Greenwoods Executive Village in Pasig City | 1.06s
[2246] 2 Bedroom Condo at One Serendra, Taguig | 1.15s
[2247] 3BR Ho

[2367] 3-Storey Townhouse in Valle Verde 6, Pasig City | 1.18s
[2368] 4 Bedroom House and Lot in Parkwood Greens, Executive Vllage, Pasig City | 0.97s
[2369] Studio Type Condominium in The Pearl Place, Pasig City | 1.19s
[2370] 2 Bedroom Condominium in East Ortigas Mansion, Pasig City | 1.24s
[2371] 4 Bedroom House and Lot in Greenwoods Executive Village, Pasig City | 1.31s
[2372] 5 Bedroom House at Casa Milan, Novaliches | 0.98s
[2373] 5 BR House in Capitol Hills, Quezon City | 1.00s
[2374] House and Lot in Capitol Golf Course | 0.97s
[2375] 5 BR Townhouse in San Juan | 1.05s
[2376] Studio Type Condominium in Avida Towers Cloverleaf, Quezon City | 1.20s
[2377] 2BR Unit in Baron Residences, San Juan City | 1.27s
[2378] 3 Bedroom Condominium in Aspire Tower, Quezon City | 1.09s
[2379] 3 Bedroom Condominium in Flair Towers - South Tower, Mandaluyong | 1.14s
[2380] 2 Bedroom Condominium in The Grove by Rockwell - Tower C, Pasig City | 1.21s
[2381] 1 Bedroom Loft-Type Condominium in The Gr

In [14]:
# print the total time it took to scrape
print('Total Number of Items: %d' % len(data))
print('Average Time per Item: %.2f' % (total_time / len(data)))
print('Total Time: %.2f' % total_time)

Total Number of Items: 2451
Average Time per Item: 1.40
Total Time: 3424.15


In [9]:
column_names = ['Name', 'Address', 'Author', 'Price', 'Bedrooms', 'Showers', 'Parking', 'Furnish', 'Total Developed', 'Lot Area', 'Features', 'Facilities', 'Nearby Places', 'URL', 'Timestamp']
df = pd.DataFrame(data, columns = column_names)
df.head()

,Name,Address,Author,Price,Bedrooms,Showers,Parking,Furnish,Total Developed,Lot Area,Features,Facilities,Nearby Places,URL,Timestamp
0,"3 BR Condo in The Grand Midori Ortigas, Pasig",,"Federal Land, Inc.","Sale: from ₱ 29,700,000",None,None,None,None,None,None,"[Ensuite, Range Hood, Stove/Burner]","[Conference Room, Entertainment Area, Function...","[School, Hospital]",https://propertyaccess.ph/property/3-br-condo-...,2022-10-17 17:39:48
1,"1 Bedroom Condo in The Grand Midori Ortigas, P...",,"Federal Land, Inc.","Sale: from ₱ 10,200,000",None,None,None,None,None,None,[Fiber ready],"[Conference Room, Entertainment Area, Function...","[School, Hospital]",https://propertyaccess.ph/property/1br-condo-i...,2022-10-17 17:39:50
2,"3 Bedroom Condo in Aurelia Residences, Taguig","McKinley Parkway, Taguig, Metro Manila",Shang Properties,"Sale: from ₱ 107,300,000",None,None,None,None,None,None,"[Central air conditioning, Balcony, Built-in w...","[Security, CCTV, Entertainment Area, Fitness C...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/3-bedroom-c...,2022-10-17 17:39:51
3,3 Bedroom Condo in Shang Residences at Wack Wa...,"Wack Wack Road, Mandaluyong, Metro Manila",Shang Properties,"Sale: from ₱ 54,500,000",None,None,None,None,None,None,"[Central air conditioning, Balcony, Built-in w...","[Security, CCTV, Club House, Entertainment Are...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/3-bedroom-c...,2022-10-17 17:39:53
4,"2BR Condo in Residences at The Galleon, Pasig",,Ortigas Land,"Sale: from ₱ 41,500,000",None,None,None,None,None,None,"[Central air conditioning, Balcony, Built-in w...","[Security, CCTV, Clubhouse, Connected to mall,...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/2br-condo-i...,2022-10-17 17:39:55


In [10]:
df.describe()

,Name,Address,Author,Price,Bedrooms,Showers,Parking,Furnish,Total Developed,Lot Area,Features,Facilities,Nearby Places,URL,Timestamp
count,2451,2451,2451,2451,0,0,0,0,0,0,2451,2451,2451,2451,2451
unique,1459,578,306,1002,0,0,0,0,0,0,620,442,11,2451,2422
top,"4 Bedroom House & Lot, Quezon City","Taguig, Metro Manila",Raimund,"Sale: ₱ 15,000,000",NaN,NaN,NaN,NaN,NaN,NaN,[],[],[],https://propertyaccess.ph/property/3-br-condo-...,2022-10-17 18:27:38
freq,53,103,162,42,NaN,NaN,NaN,NaN,NaN,NaN,165,1190,1411,1,2


In [11]:
df.to_json('unclean_data.json', orient='records', indent=2)

In [12]:
driver.close()

## Data Cleaning

In [2]:
data = pd.read_json('unclean_data.json')
data

,Name,Address,Author,Price,Bedrooms,Showers,Parking,Furnish,Total Developed,Lot Area,Features,Facilities,Nearby Places,URL,Timestamp
0,"3 Bedroom Condo in Aurelia Residences, Taguig","McKinley Parkway, Taguig, Metro Manila",Shang Properties,"Sale: from ₱ 107,300,000",3,4,2,Unfurnished,from 242 sqm,None,"[Central air conditioning, Balcony, Built-in w...","[Security, CCTV, Entertainment Area, Fitness C...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/3-bedroom-c...,2022-09-27 00:57:33
1,3 Bedroom Condo in Shang Residences at Wack Wa...,"Wack Wack Road, Mandaluyong, Metro Manila",Shang Properties,"Sale: from ₱ 54,500,000",3,4,3,Unfurnished,from 231 sqm,None,"[Central air conditioning, Balcony, Built-in w...","[Security, CCTV, Club House, Entertainment Are...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/3-bedroom-c...,2022-09-27 00:57:34
2,"2BR Condo in Residences at The Galleon, Pasig",,Ortigas Land,"Sale: from ₱ 41,500,000",2,2,2,Unfurnished,from 109 sqm,None,"[Central air conditioning, Balcony, Built-in w...","[Security, CCTV, Clubhouse, Connected to mall,...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/2br-condo-i...,2022-09-27 00:57:36
3,"Penthouse in Residences at The Galleon, Pasig","ADB Avenue, Pasig, Metro Manila",Ortigas Land,"Sale: from ₱ 111,500,000",3,4,4+,Unfurnished,from 268 sqm,None,"[Central air conditioning, Balcony, Built-in w...","[Security, CCTV, Clubhouse, Connected to mall,...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/penthouse-i...,2022-09-27 00:57:38
4,"3 Bedroom Condo in Aurelia Residences, Taguig","McKinley Parkway, Taguig, Metro Manila",Shang Properties,"Sale: from ₱ 181,302,240",3,4,3,Unfurnished,from 337 sqm,None,[Central air conditioning],"[Security, CCTV, Entertainment Area, Fitness C...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/3-bedroom-c...,2022-09-27 00:57:47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"Residential Lot, Calabarzon",Calabarzon,Alyssa Barroso,"Sale: ₱ 7,700,000",None,None,None,None,from 242 sqm,275 sqm,[],[],[],https://propertyaccess.ph/property/residential...,2022-09-27 01:00:06
96,"3 Bedroom Condo in Avida Cityflex Towers, Taguig","Taguig, Metro Manila",Christine Li,"Sale: ₱ 15,000,000",3,1,0,Furnished,76 sqm,None,"[Central air conditioning, Maid’s Room, Range ...","[Fitness Center, Garden/Lanai, Lobby, Play Are...","[School, Hospital, Mall]",https://propertyaccess.ph/property/3-bedroom-c...,2022-09-27 01:00:08
97,"Office Space, Makati","V.A. Rufino Street, Makati, Metro Manila",RBK Property Consultants Inc.,"Sale: ₱ 35,000,000",None,None,None,None,219 sqm,None,[],"[Security, Building Reception Area, CCTV, Dini...",[],https://propertyaccess.ph/property/office-spac...,2022-09-27 01:00:10
98,"Office Space, Makati","V.A. Rufino Street, Makati, Metro Manila",RBK Property Consultants Inc.,"Sale: ₱ 58,000,000",None,None,None,None,213 sqm,None,[],"[Security, Building Reception Area, CCTV, Dini...",[],https://propertyaccess.ph/property/office-spac...,2022-09-27 01:00:12


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Name             100 non-null    object        
 1   Address          100 non-null    object        
 2   Author           100 non-null    object        
 3   Price            100 non-null    object        
 4   Bedrooms         93 non-null     object        
 5   Showers          93 non-null     object        
 6   Parking          93 non-null     object        
 7   Furnish          93 non-null     object        
 8   Total Developed  100 non-null    object        
 9   Lot Area         28 non-null     object        
 10  Features         100 non-null    object        
 11  Facilities       100 non-null    object        
 12  Nearby Places    100 non-null    object        
 13  URL              100 non-null    object        
 14  Timestamp        100 non-null    datetime64

In [4]:
data['Price'] = data['Price'].str.replace('Sale: from ₱ ','')
data['Price'] = data['Price'].str.replace('Sale: ₱ ','')

In [5]:
data['Price'] = data['Price'].str.replace(',','').astype(int)

In [6]:
data['Price']

0     107300000
1      54500000
2      41500000
3     111500000
4     181302240
        ...    
95      7700000
96     15000000
97     35000000
98     58000000
99      3024000
Name: Price, Length: 100, dtype: int64

In [7]:
data['Total Developed'] = data['Total Developed'].str.replace('from','')
data['Total Developed'] = data['Total Developed'].str.replace('sqm','').astype(int)

In [8]:
data['Total Developed']

0     242
1     231
2     109
3     268
4     337
     ... 
95    242
96     76
97    219
98    213
99     52
Name: Total Developed, Length: 100, dtype: int64

In [9]:
data.head()

,Name,Address,Author,Price,Bedrooms,Showers,Parking,Furnish,Total Developed,Lot Area,Features,Facilities,Nearby Places,URL,Timestamp
0,"3 Bedroom Condo in Aurelia Residences, Taguig","McKinley Parkway, Taguig, Metro Manila",Shang Properties,107300000,3,4,2,Unfurnished,242,None,"[Central air conditioning, Balcony, Built-in w...","[Security, CCTV, Entertainment Area, Fitness C...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/3-bedroom-c...,2022-09-27 00:57:33
1,3 Bedroom Condo in Shang Residences at Wack Wa...,"Wack Wack Road, Mandaluyong, Metro Manila",Shang Properties,54500000,3,4,3,Unfurnished,231,None,"[Central air conditioning, Balcony, Built-in w...","[Security, CCTV, Club House, Entertainment Are...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/3-bedroom-c...,2022-09-27 00:57:34
2,"2BR Condo in Residences at The Galleon, Pasig",,Ortigas Land,41500000,2,2,2,Unfurnished,109,None,"[Central air conditioning, Balcony, Built-in w...","[Security, CCTV, Clubhouse, Connected to mall,...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/2br-condo-i...,2022-09-27 00:57:36
3,"Penthouse in Residences at The Galleon, Pasig","ADB Avenue, Pasig, Metro Manila",Ortigas Land,111500000,3,4,4+,Unfurnished,268,None,"[Central air conditioning, Balcony, Built-in w...","[Security, CCTV, Clubhouse, Connected to mall,...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/penthouse-i...,2022-09-27 00:57:38
4,"3 Bedroom Condo in Aurelia Residences, Taguig","McKinley Parkway, Taguig, Metro Manila",Shang Properties,181302240,3,4,3,Unfurnished,337,None,[Central air conditioning],"[Security, CCTV, Entertainment Area, Fitness C...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/3-bedroom-c...,2022-09-27 00:57:47


In [10]:
data['Timestamp'] = pd.to_datetime(data['Timestamp']).dt.date

In [11]:
data['Timestamp']

0     2022-09-27
1     2022-09-27
2     2022-09-27
3     2022-09-27
4     2022-09-27
         ...    
95    2022-09-27
96    2022-09-27
97    2022-09-27
98    2022-09-27
99    2022-09-27
Name: Timestamp, Length: 100, dtype: object

In [12]:
# Checking null values per column
data.isna().sum()

Name                0
Address             0
Author              0
Price               0
Bedrooms            7
Showers             7
Parking             7
Furnish             7
Total Developed     0
Lot Area           72
Features            0
Facilities          0
Nearby Places       0
URL                 0
Timestamp           0
dtype: int64

In [13]:
data=data.fillna(0)

In [14]:
data.isna().sum()

Name               0
Address            0
Author             0
Price              0
Bedrooms           0
Showers            0
Parking            0
Furnish            0
Total Developed    0
Lot Area           0
Features           0
Facilities         0
Nearby Places      0
URL                0
Timestamp          0
dtype: int64

In [15]:
data['Lot Area'] = data['Lot Area'].str.replace(' sqm','')

In [16]:
data['Lot Area'] = data['Lot Area'].fillna(0)

In [17]:
data['Lot Area'] = data['Lot Area'].astype(int)

In [18]:
data['Showers'] = data['Showers'].str.replace('+','')
data['Parking'] = data['Parking'].str.replace('+','')
data['Bedrooms'] = data['Bedrooms'].str.replace('+','')

/var/folders/b5/583sbyw12yqf8bss252d8ndc0000gn/T/ipykernel_46913/1857973885.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Showers'] = data['Showers'].str.replace('+','')
/var/folders/b5/583sbyw12yqf8bss252d8ndc0000gn/T/ipykernel_46913/1857973885.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Parking'] = data['Parking'].str.replace('+','')
/var/folders/b5/583sbyw12yqf8bss252d8ndc0000gn/T/ipykernel_46913/1857973885.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Bedrooms'] = data['Bedrooms'

In [19]:
data['Showers'] = data['Showers'].astype(float)
data['Parking'] = data['Parking'].astype(float)
data['Bedrooms'] = data['Bedrooms'].astype(float)
# NOTE that int cant be used because of some values with decimals
# for situations with more than 5 of these categories, + is just used. 
# So it was removed in the code above to make it into a float
# please check error mssg

ValueError: could not convert string to float: 'Studio'

In [48]:
# converting unfurnished and furnished to boolean
data['Furnish'] = data['Furnish'].replace({'Unfurnished':0,'Furnished':1,'Semi-furnished':2})
data['Furnish'] = data['Furnish'].astype(int)

In [49]:
data.dtypes

Name                object
Address             object
Author              object
Price                int64
Bedrooms           float64
Showers            float64
Parking            float64
Furnish              int64
Total Developed      int64
Lot Area             int64
Features            object
Facilities          object
Nearby Places       object
URL                 object
Timestamp           object
dtype: object

In [50]:
data.head()

,Name,Address,Author,Price,Bedrooms,Showers,Parking,Furnish,Total Developed,Lot Area,Features,Facilities,Nearby Places,URL,Timestamp
0,"3 Bedroom Condo in Aurelia Residences, Taguig","McKinley Parkway, Taguig, Metro Manila",Shang Properties,107300000,4.0,4.0,4.0,0,242,0,"[Central air conditioning, Balcony, Built-in w...","[Security, CCTV, Entertainment Area, Fitness C...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/3-bedroom-c...,2022-09-27
1,3 Bedroom Condo in Shang Residences at Wack Wa...,"Wack Wack Road, Mandaluyong, Metro Manila",Shang Properties,54500000,4.0,4.0,4.0,0,231,0,"[Central air conditioning, Balcony, Built-in w...","[Security, CCTV, Club House, Entertainment Are...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/3-bedroom-c...,2022-09-27
2,"2BR Condo in Residences at The Galleon, Pasig",,Ortigas Land,41500000,2.0,2.0,2.0,0,109,0,"[Central air conditioning, Balcony, Built-in w...","[Security, CCTV, Clubhouse, Connected to mall,...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/2br-condo-i...,2022-09-27
3,"Penthouse in Residences at The Galleon, Pasig","ADB Avenue, Pasig, Metro Manila",Ortigas Land,111500000,4.0,4.0,4.0,0,268,0,"[Central air conditioning, Balcony, Built-in w...","[Security, CCTV, Clubhouse, Connected to mall,...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/penthouse-i...,2022-09-27
4,"3 Bedroom Condo in Aurelia Residences, Taguig","McKinley Parkway, Taguig, Metro Manila",Shang Properties,181302240,4.0,4.0,4.0,0,337,0,[Central air conditioning],"[Security, CCTV, Entertainment Area, Fitness C...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/3-bedroom-c...,2022-09-27


In [51]:
data.to_json('cleaned data.json',orient='columns')

## Challenges
- Having to go to each page in order to get the necessary data (takes longer time)
- showers, parking, and bedrooms have values with 0.5 increments and for those with more than 5, it's just placed as 5+

## Conclusion
- [Does it have personal identifiable information?]
- 